In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as scs # sparse matrix construction 
import scipy.linalg as scl # linear algebra algorithms
import scipy.optimize as sco # for minimization use
import matplotlib.pylab as plt # for visualization
from cvxopt import solvers, matrix
import time


def solver(path_name):

    # In the following, the fixed_constraints are constructed from the board directly. 
    def fixed_constraints(N=9):
        rowC = np.zeros(N)
        rowC[0] =1
        rowR = np.zeros(N)
        rowR[0] =1
        row = scl.toeplitz(rowC, rowR)
        ROW = np.kron(row, np.kron(np.ones((1,N)), np.eye(N)))

        colR = np.kron(np.ones((1,N)), rowC)
        col  = scl.toeplitz(rowC, colR)
        COL  = np.kron(col, np.eye(N))

        M = int(np.sqrt(N))
        boxC = np.zeros(M)
        boxC[0]=1
        boxR = np.kron(np.ones((1, M)), boxC) 
        box = scl.toeplitz(boxC, boxR)
        box = np.kron(np.eye(M), box)
        BOX = np.kron(box, np.block([np.eye(N), np.eye(N) ,np.eye(N)]))

        cell = np.eye(N**2)
        CELL = np.kron(cell, np.ones((1,N)))

        return scs.csr_matrix(np.block([[ROW],[COL],[BOX],[CELL]]))

    # For the constraint from clues, we extract the nonzeros from the quiz string.
    def clue_constraint(input_quiz, N=9):
        m = np.reshape([int(c) for c in input_quiz], (N,N))
        r, c = np.where(m.T)
        v = np.array([m[c[d],r[d]] for d in range(len(r))])

        table = N * c + r
        table = np.block([[table],[v-1]])

        # it is faster to use lil_matrix when changing the sparse structure.
        CLUE = scs.lil_matrix((len(table.T), N**3))
        for i in range(len(table.T)):
            CLUE[i,table[0,i]*N + table[1,i]] = 1
        # change back to csr_matrix.
        CLUE = CLUE.tocsr() 

        return CLUE
    
    data=pd.read_csv(path_name) 
    solvers.options['show_progress'] = False
    corr_cnt = 0
    start = time.time()
    random_seed = 42
    np.random.seed(random_seed)


    corr_cnt = 0
    start = time.time()
    
    if len(data) > 1000:
        samples = np.random.choice(len(data), 1000)
    else:
        samples = range(len(data))     
    
    for i in range(len(samples)):
        quiz = data["quizzes"][samples[i]]
        solu = data["solutions"][samples[i]]
        A0 = fixed_constraints()
        A1 = clue_constraint(quiz)

        # Formulate the matrix A and vector B (B is all ones).
        A = scs.vstack((A0,A1))
        A = A.toarray()
        B = np.ones(A.shape[0])
        # Because rank defficiency. We need to extract effective rank.
        u, s, vh = np.linalg.svd(A, full_matrices=False)
        K = np.sum(s > 1e-12)
        S_ = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
        A = S_@vh
        B = u.T@B
        B = B[:K]

        c = matrix(np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ]))
        G = matrix(np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                             [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]]))
        h = matrix(np.zeros(A.shape[1]*2))
        H = matrix(np.block([A, -A]))
        b = matrix(B)

        sol = solvers.lp(c,G,h,H,b)

        # postprocessing the solution
        X = np.array(sol['x']).T[0]
        x = X[:A.shape[1]] - X[A.shape[1]:]

        # map to board
        z = np.reshape(x, (81, 9))
        if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
                          - np.reshape([int(c) for c in solu], (9,9)), np.inf) >0:
            pass
        else:
            #print("CORRECT")
            corr_cnt += 1
        if (i+1) % 5 == 0:
            end = time.time()
            print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

    end = time.time()
    print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )
    return corr_cnt/(i+1)

In [23]:
path1="C:/Users/Junya/Desktop/Math110B/Project1/Project-1-Sudoku/input/small1.csv"

In [24]:
solver(path1)

Aver Time:   1.09 secs. Success rate: 5 / 5 
Aver Time:   1.15 secs. Success rate: 10 / 10 
Aver Time:   1.20 secs. Success rate: 15 / 15 
Aver Time:   1.21 secs. Success rate: 20 / 20 
Aver Time:   1.20 secs. Success rate: 24 / 24 


1.0

In [25]:
path3="C:/Users/Junya/Desktop/Math110B/Project1/Project-1-Sudoku/input/large1.csv"

In [26]:
solver(path3)

Aver Time:   1.40 secs. Success rate: 3 / 5 
Aver Time:   1.34 secs. Success rate: 7 / 10 
Aver Time:   1.35 secs. Success rate: 12 / 15 
Aver Time:   1.32 secs. Success rate: 16 / 20 
Aver Time:   1.30 secs. Success rate: 20 / 25 
Aver Time:   1.33 secs. Success rate: 24 / 30 
Aver Time:   1.36 secs. Success rate: 28 / 35 
Aver Time:   1.39 secs. Success rate: 32 / 40 
Aver Time:   1.40 secs. Success rate: 36 / 45 
Aver Time:   1.39 secs. Success rate: 40 / 50 
Aver Time:   1.39 secs. Success rate: 43 / 55 
Aver Time:   1.39 secs. Success rate: 47 / 60 
Aver Time:   1.38 secs. Success rate: 51 / 65 
Aver Time:   1.39 secs. Success rate: 56 / 70 
Aver Time:   1.39 secs. Success rate: 59 / 75 
Aver Time:   1.40 secs. Success rate: 64 / 80 
Aver Time:   1.40 secs. Success rate: 68 / 85 
Aver Time:   1.39 secs. Success rate: 70 / 90 
Aver Time:   1.39 secs. Success rate: 74 / 95 
Aver Time:   1.39 secs. Success rate: 78 / 100 
Aver Time:   1.38 secs. Success rate: 82 / 105 
Aver Time:   1

Aver Time:   1.38 secs. Success rate: 693 / 850 
Aver Time:   1.38 secs. Success rate: 697 / 855 
Aver Time:   1.38 secs. Success rate: 701 / 860 
Aver Time:   1.38 secs. Success rate: 706 / 865 
Aver Time:   1.38 secs. Success rate: 711 / 870 
Aver Time:   1.39 secs. Success rate: 715 / 875 
Aver Time:   1.38 secs. Success rate: 719 / 880 
Aver Time:   1.38 secs. Success rate: 722 / 885 
Aver Time:   1.38 secs. Success rate: 727 / 890 
Aver Time:   1.38 secs. Success rate: 732 / 895 
Aver Time:   1.38 secs. Success rate: 736 / 900 
Aver Time:   1.38 secs. Success rate: 740 / 905 
Aver Time:   1.38 secs. Success rate: 744 / 910 
Aver Time:   1.38 secs. Success rate: 749 / 915 
Aver Time:   1.38 secs. Success rate: 752 / 920 
Aver Time:   1.39 secs. Success rate: 757 / 925 
Aver Time:   1.39 secs. Success rate: 758 / 930 
Aver Time:   1.39 secs. Success rate: 763 / 935 
Aver Time:   1.39 secs. Success rate: 767 / 940 
Aver Time:   1.39 secs. Success rate: 771 / 945 
Aver Time:   1.39 se

0.818

In [27]:
path2="C:/Users/Junya/Desktop/Math110B/Project1/Project-1-Sudoku/input/small2.csv"

In [28]:
solver(path2)

Aver Time:   1.39 secs. Success rate: 2 / 5 
Aver Time:   1.36 secs. Success rate: 3 / 10 
Aver Time:   1.35 secs. Success rate: 4 / 15 
Aver Time:   1.34 secs. Success rate: 5 / 20 
Aver Time:   1.32 secs. Success rate: 7 / 25 
Aver Time:   1.30 secs. Success rate: 7 / 30 
Aver Time:   1.29 secs. Success rate: 8 / 35 
Aver Time:   1.28 secs. Success rate: 11 / 40 
Aver Time:   1.28 secs. Success rate: 14 / 45 
Aver Time:   1.29 secs. Success rate: 14 / 50 
Aver Time:   1.30 secs. Success rate: 14 / 55 
Aver Time:   1.30 secs. Success rate: 15 / 60 
Aver Time:   1.31 secs. Success rate: 19 / 65 
Aver Time:   1.31 secs. Success rate: 22 / 70 
Aver Time:   1.31 secs. Success rate: 23 / 75 
Aver Time:   1.30 secs. Success rate: 25 / 80 
Aver Time:   1.31 secs. Success rate: 29 / 85 
Aver Time:   1.31 secs. Success rate: 30 / 90 
Aver Time:   1.31 secs. Success rate: 33 / 95 
Aver Time:   1.31 secs. Success rate: 35 / 100 
Aver Time:   1.31 secs. Success rate: 37 / 105 
Aver Time:   1.31 s

Aver Time:   1.47 secs. Success rate: 289 / 850 
Aver Time:   1.47 secs. Success rate: 291 / 855 
Aver Time:   1.47 secs. Success rate: 293 / 860 
Aver Time:   1.47 secs. Success rate: 295 / 865 
Aver Time:   1.48 secs. Success rate: 295 / 870 
Aver Time:   1.48 secs. Success rate: 298 / 875 
Aver Time:   1.48 secs. Success rate: 299 / 880 
Aver Time:   1.48 secs. Success rate: 300 / 885 
Aver Time:   1.48 secs. Success rate: 301 / 890 
Aver Time:   1.49 secs. Success rate: 302 / 895 
Aver Time:   1.49 secs. Success rate: 303 / 900 
Aver Time:   1.49 secs. Success rate: 303 / 905 
Aver Time:   1.49 secs. Success rate: 303 / 910 
Aver Time:   1.49 secs. Success rate: 303 / 915 
Aver Time:   1.49 secs. Success rate: 303 / 920 
Aver Time:   1.49 secs. Success rate: 306 / 925 
Aver Time:   1.50 secs. Success rate: 308 / 930 
Aver Time:   1.50 secs. Success rate: 309 / 935 
Aver Time:   1.50 secs. Success rate: 310 / 940 
Aver Time:   1.50 secs. Success rate: 312 / 945 
Aver Time:   1.50 se

0.324

In [29]:
path4="C:/Users/Junya/Desktop/Math110B/Project1/Project-1-Sudoku/input/large2.csv"

In [30]:
solver(path4)

Aver Time:   1.25 secs. Success rate: 5 / 5 
Aver Time:   1.24 secs. Success rate: 10 / 10 
Aver Time:   1.19 secs. Success rate: 15 / 15 
Aver Time:   1.18 secs. Success rate: 20 / 20 
Aver Time:   1.19 secs. Success rate: 25 / 25 
Aver Time:   1.18 secs. Success rate: 30 / 30 
Aver Time:   1.17 secs. Success rate: 35 / 35 
Aver Time:   1.17 secs. Success rate: 40 / 40 
Aver Time:   1.17 secs. Success rate: 45 / 45 
Aver Time:   1.17 secs. Success rate: 50 / 50 
Aver Time:   1.17 secs. Success rate: 55 / 55 
Aver Time:   1.17 secs. Success rate: 60 / 60 
Aver Time:   1.17 secs. Success rate: 65 / 65 
Aver Time:   1.16 secs. Success rate: 70 / 70 
Aver Time:   1.16 secs. Success rate: 75 / 75 
Aver Time:   1.17 secs. Success rate: 80 / 80 
Aver Time:   1.17 secs. Success rate: 85 / 85 
Aver Time:   1.17 secs. Success rate: 90 / 90 
Aver Time:   1.17 secs. Success rate: 95 / 95 
Aver Time:   1.18 secs. Success rate: 100 / 100 
Aver Time:   1.18 secs. Success rate: 105 / 105 
Aver Time: 

Aver Time:   1.43 secs. Success rate: 850 / 850 
Aver Time:   1.43 secs. Success rate: 855 / 855 
Aver Time:   1.43 secs. Success rate: 860 / 860 
Aver Time:   1.43 secs. Success rate: 865 / 865 
Aver Time:   1.43 secs. Success rate: 870 / 870 
Aver Time:   1.43 secs. Success rate: 875 / 875 
Aver Time:   1.43 secs. Success rate: 880 / 880 
Aver Time:   1.43 secs. Success rate: 885 / 885 
Aver Time:   1.43 secs. Success rate: 890 / 890 
Aver Time:   1.43 secs. Success rate: 895 / 895 
Aver Time:   1.43 secs. Success rate: 900 / 900 
Aver Time:   1.43 secs. Success rate: 905 / 905 
Aver Time:   1.44 secs. Success rate: 910 / 910 
Aver Time:   1.44 secs. Success rate: 915 / 915 
Aver Time:   1.44 secs. Success rate: 920 / 920 
Aver Time:   1.44 secs. Success rate: 925 / 925 
Aver Time:   1.44 secs. Success rate: 930 / 930 
Aver Time:   1.44 secs. Success rate: 935 / 935 
Aver Time:   1.44 secs. Success rate: 940 / 940 
Aver Time:   1.44 secs. Success rate: 945 / 945 
Aver Time:   1.44 se

1.0